In [1]:

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [2]:
def calculate_length(a,b):
    a = np.array(a)
    b = np.array(b)
    
    length = math.sqrt((a[1]-b[1])**2+(a[2]-b[2])**2)
    
    return length

In [3]:
#腳板翹高
#Q1會因為拍攝角度會有十幾度的差距問題 可能不能用夾角公式做計算
#Q2偵測前腳板抬起的話 在走路的過程也會一併被算進去
import cv2
import mediapipe as mp
import numpy as np
import random
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
# cap.release()
# cv2.destroyAllWindows()

# def calculate_length(a,b):
#     a = np.array(a)
#     b = np.array(b)
    
#     length = math.sqrt((a[1]-b[1])**2+(a[2]-b[2])**2)
    
#     return length
def calculate_length(a,b):
    a = np.array(a)
    b = np.array(b)
    
    length = math.sqrt((a[1]-b[1])**2+(a[2]-b[2])**2)
    print('length:', length)
    
    return length

cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None
distance = 0
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            landmark1 = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            landmark2 = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            print('landmark1:',landmark1, 'landmark2',landmark2)

            a = np.array(landmark1)
            b = np.array(landmark2)

            distance = math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)
            print('distance:', distance)
#             distance = calculate_length(landmark1, landmark2)
#             print('distance',distance)
            
#         
            
            # Visualize angle
            cv2.putText(image, str(distance), 
                           tuple(np.multiply(landmark2, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            
            # Curl counter logic
            if distance < 20 and distance > 10:
                stage = "down"
                time = 0
            if distance < 10 and distance > 0 and stage =='down':
                stage = "up"
                counter += 1
            


        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (305,73), (245,117,16), -1)
        
#         # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
#         # Stage data
        cv2.putText(image, 'STAGE', (95,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
#         cv2.putText(image, stage, 
#                     (80,60), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image,f"Max Distance: {distance:.2f}",(10 ,180),
                    cv2.FONT_HERSHEY_SIMPLEX ,1,(255 ,255 ,255) ,2 ,cv2.LINE_AA)
        
        
#         # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )              
#         
        cv2.imshow('Mediapipe Feed', image)
#         cv2.imshow('MediaPipe Hands', image)
 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

landmark1: [0.6215739846229553, 2.4521892070770264] landmark2 [0.613795816898346, 1.884284496307373]
distance: 0.5679579741561128
landmark1: [0.5949999094009399, 2.4765894412994385] landmark2 [0.6064518690109253, 1.8913370370864868]
distance: 0.5853644369245102
landmark1: [0.5832955837249756, 2.486241102218628] landmark2 [0.5988258719444275, 1.8938499689102173]
distance: 0.5925946714868455
landmark1: [0.5821084380149841, 2.485414981842041] landmark2 [0.5966100096702576, 1.8942774534225464]
distance: 0.5913153753170822
landmark1: [0.5796197652816772, 2.483787775039673] landmark2 [0.5942343473434448, 1.8941388130187988]
distance: 0.589830047065368
landmark1: [0.5764889717102051, 2.4833407402038574] landmark2 [0.5916116237640381, 1.8940173387527466]
distance: 0.5895174010180263
landmark1: [0.579277515411377, 2.486736297607422] landmark2 [0.5923441648483276, 1.8945086002349854]
distance: 0.5923718282148185
landmark1: [0.5810582637786865, 2.487273931503296] landmark2 [0.5925533771514893, 1.